In [1]:
import os.path as osp
import random 
import os

import torch
import torch.nn.functional as F
from torch import optim, nn

import pandas as pd
import networkx as nx
import numpy as np

from torch_geometric import data as DATA
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import k_hop_subgraph, to_scipy_sparse_matrix
from torch_geometric.data import Data, InMemoryDataset
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.utils import negative_sampling

from scipy.sparse.csgraph import shortest_path
from itertools import chain

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
effort = "GAT_DOC2Vec"

np.random.seed(0)
random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=False),
])

In [4]:
edge_folds = pd.read_csv("./data_split.csv", header=0)
edge_folds = edge_folds[edge_folds['Label'] == 1]
edge_folds.loc[edge_folds['Fold'] == 'Test', 'Fold'] = 5
edge_folds.loc[edge_folds['Fold'] == '3', 'Fold'] = 3
edge_folds.loc[edge_folds['Fold'] == '4', 'Fold'] = 4
#edge_folds['Node1'] = edge_folds['Node1'].astype(int)
#edge_folds['Node2'] = edge_folds['Node2'].astype(int)

/home/rf27/miniconda3/envs/New_awesome_environment/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()

        self.conv1 = GATConv(in_channels, 2 * hidden_channels, heads=2, dropout=0.6)

        self.conv2 = GATConv(4 * hidden_channels, hidden_channels, heads=1, concat=False,
                             dropout=0.6)
        
        self.conv3 = GATConv(hidden_channels, out_channels, heads=1, concat=False,
                             dropout=0.6)
        
        self.fc1 = nn.Linear(out_channels * 2, out_channels)
        self.fc2 = nn.Linear(out_channels, 1)

    def encode(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv2(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv3(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        return x
    
    def decode(self, z, edge_label_index):
        x = torch.cat((z[edge_label_index[0]], z[edge_label_index[1]]), 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.squeeze(1)
        return x

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, out_channels)
        
        self.fc1 = nn.Linear(out_channels * 2, out_channels)
        self.fc2 = nn.Linear(out_channels, 1)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        return self.conv3(x, edge_index)

    def decode(self, z, edge_label_index):
        x = torch.cat((z[edge_label_index[0]], z[edge_label_index[1]]), 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.squeeze(1)
        return x
    #def decode_all(self, z):
    #    prob_adj = z @ z.t()
    #    return (prob_adj > 0).nonzero(as_tuple=False).t()

In [6]:
def train(train_data):
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out.cpu(), edge_label)
    loss.backward()
    optimizer.step()
    return loss

In [7]:
@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    
    # We perform a new round of negative sampling for every validation:
    neg_edge_index = negative_sampling(
        edge_index = data.edge_index, num_nodes=data.num_nodes,
        num_neg_samples = data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    
    edge_label = torch.cat([
        data.edge_label,
        data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)
    
    out = model.decode(z, edge_label_index).view(-1).sigmoid()
    return roc_auc_score(edge_label.cpu().numpy(), out.cpu().numpy())

In [8]:
'''
edgeframe = pd.read_csv('./Training/training_graph.csv', 
                        sep=',', header=0, names=['source', 'dest'])
edgeframe = edgeframe[edgeframe['source'] != edgeframe['dest']]

G = nx.from_pandas_edgelist(edgeframe, 'source', 'dest')

edge_index = []
for e1, e2 in G.edges:
    edge_index.append([e1, e2])

edge_index
'''

"\nedgeframe = pd.read_csv('./Training/training_graph.csv', \n                        sep=',', header=0, names=['source', 'dest'])\nedgeframe = edgeframe[edgeframe['source'] != edgeframe['dest']]\n\nG = nx.from_pandas_edgelist(edgeframe, 'source', 'dest')\n\nedge_index = []\nfor e1, e2 in G.edges:\n    edge_index.append([e1, e2])\n\nedge_index\n"

In [9]:
# Text type features
features = pd.read_csv("./GLoVE_features.csv")
#features = pd.DataFrame(data={'id': range(edge_folds.shape[0])}) # THIS IF YOU WANT TO INCLUDE NOTHING
features = features.sort_values(by=['id'])

# PCA ONLY on TF_IDF

# Page type features
page_type = pd.read_csv("./Training/node_classification.csv")
myEncoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
myEncoder.fit(page_type['page_type'].to_numpy().reshape(-1, 1))

page_type = pd.concat([page_type.drop('page_type', 1),
            pd.DataFrame(myEncoder.transform(page_type['page_type'].to_numpy().reshape(-1, 1)))], 
                         axis=1).reindex()

# Node Id Embedding features
embedding = torch.nn.Embedding(page_type.shape[0], 32)
embedding = pd.DataFrame(np.array(embedding.weight.data))
embedding['id'] = range(page_type.shape[0])

# Finalize Feature Embedding
features = features.merge(page_type, how='inner', on='id').merge(embedding, how='inner', on='id')
features = features.drop(columns=['id'])

features = torch.Tensor(features.to_numpy()).to(device)

In [10]:
for r in range(1, 6):
    
    with open('./logs/' + effort + '_fold' + str(r) +'.txt', 'w') as f:
        f.write("New training bitches!!\n")
    
    print("Split #", r)
    with open('./logs/' + effort + '_fold' + str(r) + '.txt', 'a') as f:
        f.write("Split # {}\n".format(r))
        
    edge_folds_train = edge_folds[edge_folds['Fold'] != r]
    edge_folds_val = edge_folds[edge_folds['Fold'] == r]
    G = nx.from_pandas_edgelist(edge_folds_train, 'Node1', 'Node2')

    edge_index = []
    for e1, e2 in G.edges:
        edge_index.append([e1, e2])
    
    G_val = nx.from_pandas_edgelist(edge_folds_val, 'Node1', 'Node2')
    edge_label_index_val = []
    for e1, e2 in G_val.edges:
        edge_label_index_val.append([e1, e2])
    
    train_graph = DATA.Data(
        x = features.to(device),
        edge_index = torch.LongTensor(edge_index).transpose(1, 0).to(device),
        edge_label_index = torch.LongTensor(edge_index).transpose(1, 0).to(device),
        edge_label = torch.ones(len(edge_index))
    )
    
    val_graph = DATA.Data(
        x = features.to(device),
        edge_index = torch.LongTensor(edge_index).transpose(1, 0).to(device),
        edge_label_index = torch.LongTensor(edge_label_index_val).transpose(1, 0).to(device),
        edge_label = torch.ones(len(edge_label_index_val))
    )
    
    model = GAT(164, 128, 64).to(device) # Change number of features accordingly
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
    criterion = torch.nn.BCEWithLogitsLoss()
    
    best_val_auc = 0
    for epoch in range(1, 1001):
        loss = train(train_graph)
        print('Train epoch: {}, Total Loss: {:.4f}'.format(epoch, loss))
        with open('./logs/' + effort + '_fold' + str(r) + '.txt', 'a') as f:
            f.write('Train epoch: {}, Total Loss: {:.4f}\n'.format(epoch, loss))  
        val_auc = test(val_graph)
        print('Total Validation AUC: {:.4f}'.format(val_auc))
        with open('./logs/' + effort + '_fold' + str(r) + '.txt', 'a') as f:
            f.write('Total Validation AUC: {:.4f}'.format(val_auc))  
        if val_auc > best_val_auc:
            best_val = val_auc
            torch.save(model.state_dict(), 
                       './models/' + effort + '_fold' + str(r) + '.pt')

Split # 1
Train epoch: 1, Total Loss: 0.7494
Total Validation AUC: 0.6631
Train epoch: 2, Total Loss: 0.6882
Total Validation AUC: 0.6321
Train epoch: 3, Total Loss: 0.7227
Total Validation AUC: 0.5548
Train epoch: 4, Total Loss: 0.7747
Total Validation AUC: 0.6876
Train epoch: 5, Total Loss: 0.7045
Total Validation AUC: 0.6667
Train epoch: 6, Total Loss: 0.6910
Total Validation AUC: 0.6060
Train epoch: 7, Total Loss: 0.6849
Total Validation AUC: 0.5592
Train epoch: 8, Total Loss: 0.6825
Total Validation AUC: 0.5409
Train epoch: 9, Total Loss: 0.6845
Total Validation AUC: 0.5284
Train epoch: 10, Total Loss: 0.6822
Total Validation AUC: 0.5078
Train epoch: 11, Total Loss: 0.6798
Total Validation AUC: 0.4959
Train epoch: 12, Total Loss: 0.6776
Total Validation AUC: 0.4888
Train epoch: 13, Total Loss: 0.6785
Total Validation AUC: 0.4935
Train epoch: 14, Total Loss: 0.6701
Total Validation AUC: 0.4876
Train epoch: 15, Total Loss: 0.6664
Total Validation AUC: 0.4845
Train epoch: 16, Total L

Total Validation AUC: 0.8886
Train epoch: 128, Total Loss: 0.4638
Total Validation AUC: 0.8860
Train epoch: 129, Total Loss: 0.4714
Total Validation AUC: 0.8858
Train epoch: 130, Total Loss: 0.4696
Total Validation AUC: 0.8888
Train epoch: 131, Total Loss: 0.4657
Total Validation AUC: 0.8883
Train epoch: 132, Total Loss: 0.4644
Total Validation AUC: 0.8861
Train epoch: 133, Total Loss: 0.4637
Total Validation AUC: 0.8899
Train epoch: 134, Total Loss: 0.4641
Total Validation AUC: 0.8939
Train epoch: 135, Total Loss: 0.4694
Total Validation AUC: 0.8921
Train epoch: 136, Total Loss: 0.4607
Total Validation AUC: 0.8921
Train epoch: 137, Total Loss: 0.4641
Total Validation AUC: 0.8923
Train epoch: 138, Total Loss: 0.4586
Total Validation AUC: 0.8941
Train epoch: 139, Total Loss: 0.4638
Total Validation AUC: 0.8925
Train epoch: 140, Total Loss: 0.4618
Total Validation AUC: 0.8945
Train epoch: 141, Total Loss: 0.4578
Total Validation AUC: 0.8952
Train epoch: 142, Total Loss: 0.4554
Total Vali

Total Validation AUC: 0.9328
Train epoch: 253, Total Loss: 0.4200
Total Validation AUC: 0.9332
Train epoch: 254, Total Loss: 0.4190
Total Validation AUC: 0.9348
Train epoch: 255, Total Loss: 0.4211
Total Validation AUC: 0.9339
Train epoch: 256, Total Loss: 0.4230
Total Validation AUC: 0.9328
Train epoch: 257, Total Loss: 0.4216
Total Validation AUC: 0.9348
Train epoch: 258, Total Loss: 0.4159
Total Validation AUC: 0.9359
Train epoch: 259, Total Loss: 0.4212
Total Validation AUC: 0.9336
Train epoch: 260, Total Loss: 0.4212
Total Validation AUC: 0.9328
Train epoch: 261, Total Loss: 0.4135
Total Validation AUC: 0.9321
Train epoch: 262, Total Loss: 0.4290
Total Validation AUC: 0.9332
Train epoch: 263, Total Loss: 0.4190
Total Validation AUC: 0.9340
Train epoch: 264, Total Loss: 0.4246
Total Validation AUC: 0.9339
Train epoch: 265, Total Loss: 0.4179
Total Validation AUC: 0.9338
Train epoch: 266, Total Loss: 0.4132
Total Validation AUC: 0.9359
Train epoch: 267, Total Loss: 0.4209
Total Vali

Total Validation AUC: 0.9468
Train epoch: 378, Total Loss: 0.4063
Total Validation AUC: 0.9491
Train epoch: 379, Total Loss: 0.4003
Total Validation AUC: 0.9482
Train epoch: 380, Total Loss: 0.4020
Total Validation AUC: 0.9468
Train epoch: 381, Total Loss: 0.4085
Total Validation AUC: 0.9448
Train epoch: 382, Total Loss: 0.4098
Total Validation AUC: 0.9466
Train epoch: 383, Total Loss: 0.4071
Total Validation AUC: 0.9472
Train epoch: 384, Total Loss: 0.4084
Total Validation AUC: 0.9464
Train epoch: 385, Total Loss: 0.4069
Total Validation AUC: 0.9453
Train epoch: 386, Total Loss: 0.4031
Total Validation AUC: 0.9469
Train epoch: 387, Total Loss: 0.4083
Total Validation AUC: 0.9488
Train epoch: 388, Total Loss: 0.4082
Total Validation AUC: 0.9468
Train epoch: 389, Total Loss: 0.3999
Total Validation AUC: 0.9480
Train epoch: 390, Total Loss: 0.4023
Total Validation AUC: 0.9463
Train epoch: 391, Total Loss: 0.4040
Total Validation AUC: 0.9480
Train epoch: 392, Total Loss: 0.4055
Total Vali

Total Validation AUC: 0.9520
Train epoch: 503, Total Loss: 0.3949
Total Validation AUC: 0.9534
Train epoch: 504, Total Loss: 0.3968
Total Validation AUC: 0.9536
Train epoch: 505, Total Loss: 0.3950
Total Validation AUC: 0.9515
Train epoch: 506, Total Loss: 0.3951
Total Validation AUC: 0.9530
Train epoch: 507, Total Loss: 0.3955
Total Validation AUC: 0.9532
Train epoch: 508, Total Loss: 0.3906
Total Validation AUC: 0.9523
Train epoch: 509, Total Loss: 0.3930
Total Validation AUC: 0.9508
Train epoch: 510, Total Loss: 0.3910
Total Validation AUC: 0.9524
Train epoch: 511, Total Loss: 0.3927
Total Validation AUC: 0.9514
Train epoch: 512, Total Loss: 0.3901
Total Validation AUC: 0.9529
Train epoch: 513, Total Loss: 0.3942
Total Validation AUC: 0.9535
Train epoch: 514, Total Loss: 0.3943
Total Validation AUC: 0.9529
Train epoch: 515, Total Loss: 0.3953
Total Validation AUC: 0.9522
Train epoch: 516, Total Loss: 0.3921
Total Validation AUC: 0.9524
Train epoch: 517, Total Loss: 0.3924
Total Vali

...

Total Validation AUC: 0.9516
Train epoch: 378, Total Loss: 0.4021
Total Validation AUC: 0.9522
Train epoch: 379, Total Loss: 0.3952
Total Validation AUC: 0.9531
Train epoch: 380, Total Loss: 0.4035
Total Validation AUC: 0.9533
Train epoch: 381, Total Loss: 0.3956
Total Validation AUC: 0.9530
Train epoch: 382, Total Loss: 0.3991
Total Validation AUC: 0.9532
Train epoch: 383, Total Loss: 0.3976
Total Validation AUC: 0.9537
Train epoch: 384, Total Loss: 0.3982
Total Validation AUC: 0.9523
Train epoch: 385, Total Loss: 0.3943
Total Validation AUC: 0.9519
Train epoch: 386, Total Loss: 0.3985
Total Validation AUC: 0.9514
Train epoch: 387, Total Loss: 0.3972
Total Validation AUC: 0.9532
Train epoch: 388, Total Loss: 0.3881
Total Validation AUC: 0.9525
Train epoch: 389, Total Loss: 0.3962
Total Validation AUC: 0.9534
Train epoch: 390, Total Loss: 0.3949
Total Validation AUC: 0.9540
Train epoch: 391, Total Loss: 0.3974
Total Validation AUC: 0.9551
Train epoch: 392, Total Loss: 0.3982
Total Vali

Total Validation AUC: 0.9555
Train epoch: 503, Total Loss: 0.3929
Total Validation AUC: 0.9547
Train epoch: 504, Total Loss: 0.3915
Total Validation AUC: 0.9545
Train epoch: 505, Total Loss: 0.3911
Total Validation AUC: 0.9562
Train epoch: 506, Total Loss: 0.3923
Total Validation AUC: 0.9556
Train epoch: 507, Total Loss: 0.3944
Total Validation AUC: 0.9551
Train epoch: 508, Total Loss: 0.3970
Total Validation AUC: 0.9558
Train epoch: 509, Total Loss: 0.3858
Total Validation AUC: 0.9559
Train epoch: 510, Total Loss: 0.3959
Total Validation AUC: 0.9552
Train epoch: 511, Total Loss: 0.3910
Total Validation AUC: 0.9550
Train epoch: 512, Total Loss: 0.3877
Total Validation AUC: 0.9560
Train epoch: 513, Total Loss: 0.3922
Total Validation AUC: 0.9547
Train epoch: 514, Total Loss: 0.3894
Total Validation AUC: 0.9547
Train epoch: 515, Total Loss: 0.3976
Total Validation AUC: 0.9538
Train epoch: 516, Total Loss: 0.3863
Total Validation AUC: 0.9557
Train epoch: 517, Total Loss: 0.3952
Total Vali

Total Validation AUC: 0.9564
Train epoch: 628, Total Loss: 0.3901
Total Validation AUC: 0.9573
Train epoch: 629, Total Loss: 0.3813
Total Validation AUC: 0.9583
Train epoch: 630, Total Loss: 0.3833
Total Validation AUC: 0.9574
Train epoch: 631, Total Loss: 0.3856
Total Validation AUC: 0.9589
Train epoch: 632, Total Loss: 0.3847
Total Validation AUC: 0.9560
Train epoch: 633, Total Loss: 0.3822
Total Validation AUC: 0.9575
Train epoch: 634, Total Loss: 0.3870
Total Validation AUC: 0.9577
Train epoch: 635, Total Loss: 0.3820
Total Validation AUC: 0.9584
Train epoch: 636, Total Loss: 0.3833
Total Validation AUC: 0.9572
Train epoch: 637, Total Loss: 0.3901
Total Validation AUC: 0.9569
Train epoch: 638, Total Loss: 0.3840
Total Validation AUC: 0.9560
Train epoch: 639, Total Loss: 0.3872
Total Validation AUC: 0.9559
Train epoch: 640, Total Loss: 0.3797
Total Validation AUC: 0.9577
Train epoch: 641, Total Loss: 0.3767
Total Validation AUC: 0.9575
Train epoch: 642, Total Loss: 0.3859
Total Vali

Total Validation AUC: 0.9590
Train epoch: 753, Total Loss: 0.3814
Total Validation AUC: 0.9580
Train epoch: 754, Total Loss: 0.3836
Total Validation AUC: 0.9588
Train epoch: 755, Total Loss: 0.3820
Total Validation AUC: 0.9573
Train epoch: 756, Total Loss: 0.3846
Total Validation AUC: 0.9586
Train epoch: 757, Total Loss: 0.3796
Total Validation AUC: 0.9588
Train epoch: 758, Total Loss: 0.3820
Total Validation AUC: 0.9594
Train epoch: 759, Total Loss: 0.3794
Total Validation AUC: 0.9587
Train epoch: 760, Total Loss: 0.3821
Total Validation AUC: 0.9579
Train epoch: 761, Total Loss: 0.3826
Total Validation AUC: 0.9589
Train epoch: 762, Total Loss: 0.3772
Total Validation AUC: 0.9580
Train epoch: 763, Total Loss: 0.3775
Total Validation AUC: 0.9589
Train epoch: 764, Total Loss: 0.3847
Total Validation AUC: 0.9577
Train epoch: 765, Total Loss: 0.3799
Total Validation AUC: 0.9583
Train epoch: 766, Total Loss: 0.3826
Total Validation AUC: 0.9585
Train epoch: 767, Total Loss: 0.3832
Total Vali

Total Validation AUC: 0.9591
Train epoch: 878, Total Loss: 0.3779
Total Validation AUC: 0.9595
Train epoch: 879, Total Loss: 0.3837
Total Validation AUC: 0.9587
Train epoch: 880, Total Loss: 0.3865
Total Validation AUC: 0.9587
Train epoch: 881, Total Loss: 0.3797
Total Validation AUC: 0.9577
Train epoch: 882, Total Loss: 0.3848
Total Validation AUC: 0.9580
Train epoch: 883, Total Loss: 0.3876
Total Validation AUC: 0.9598
Train epoch: 884, Total Loss: 0.3787
Total Validation AUC: 0.9585
Train epoch: 885, Total Loss: 0.3784
Total Validation AUC: 0.9585
Train epoch: 886, Total Loss: 0.3806
Total Validation AUC: 0.9583
Train epoch: 887, Total Loss: 0.3784
Total Validation AUC: 0.9577
Train epoch: 888, Total Loss: 0.3848
Total Validation AUC: 0.9601
Train epoch: 889, Total Loss: 0.3799
Total Validation AUC: 0.9580
Train epoch: 890, Total Loss: 0.3787
Total Validation AUC: 0.9579
Train epoch: 891, Total Loss: 0.3785
Total Validation AUC: 0.9568
Train epoch: 892, Total Loss: 0.3755
Total Vali